# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data = pd.read_csv("../output_data/cities.csv")
output_data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodinsk,58.607673,99.177909,-0.65,88,79,9.51,RU,1612932782
1,1,sorvag,62.071293,-7.305866,30.20,74,82,3.44,FO,1612932782
2,2,iralaya,14.997464,-83.225170,76.37,81,12,8.01,HN,1612932782
3,3,isangel,-19.541720,169.280350,82.54,66,2,12.15,VU,1612932783
4,4,okato,-39.191082,173.879984,64.00,76,96,8.99,NZ,1612932783
...,...,...,...,...,...,...,...,...,...,...
63,65,port alfred,-33.586407,26.885145,70.00,95,1,2.84,ZA,1612932792
64,66,saint-philippe,-21.358118,55.764656,84.20,54,0,9.22,RE,1612932792
65,67,lockhart,29.884944,-97.670000,52.00,76,90,11.50,US,1612932792
66,68,nantucket,41.283470,-70.099451,33.01,93,90,14.97,US,1612932765


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = output_data[["Latitude", "Longitude"]].astype(float)
humidity = output_data['Humidity'].astype(float)

In [5]:
fig = gmaps.figure(center=(10,20), zoom_level=.7)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_output = output_data.loc[((output_data['Max Temp'] > 40) & 
                                (output_data['Max Temp'] < 80) &
                                (output_data['Humidity'] < 60))]
ideal_output

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,busselton,-33.653158,115.345489,69.01,57,2,5.99,AU,1612932783
17,17,bluff,37.284445,-109.551788,42.80,49,100,14.97,US,1612932785
67,69,barstow,34.895796,-117.017283,60.80,29,1,11.50,US,1612932548


In [7]:
ideal_locations = ideal_output[["Latitude", "Longitude"]].astype(float)
ideal_humidity = ideal_output['Humidity'].astype(float)

In [8]:
new_fig = gmaps.figure(center=(10,20), zoom_level=.7)

ideal_heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

new_fig.add_layer(ideal_heat_layer)

new_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
cities = []

hotel_df = pd.DataFrame({'Hotel Name':cities})
hotel_df['Latitude'] = ""
hotel_df['Longitude'] = ""
hotel_df['City'] = ""
hotel_df['Country'] = ""
hotel_df.reset_index()
hotel_df

,Hotel Name,Latitude,Longitude,City,Country


In [10]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

lat = -33.653158
lng = 115.345489

params['location'] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)

hotel = name_address.json()

print(hotel['results'][0]['geometry']['location']['lat'])
print(hotel['results'][0]['geometry']['location']['lng'])
hotel['results'][0]

-33.6463929
115.3486762


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -33.6463929, 'lng': 115.3486762},
  'viewport': {'northeast': {'lat': -33.64506972010727,
    'lng': 115.3498695298927},
   'southwest': {'lat': -33.64776937989271, 'lng': 115.3471698701073}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Observatory Guest House',
 'photos': [{'height': 3264,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112137862724320262966">Observatory Guest House</a>'],
   'photo_reference': 'ATtYBwJGxJEr0O6luISBzGJZMXrzhMb5Glqv_9hWJHTQNX4Gj4aDV7GNTDtQ2TJo2oXVLQ8kiD65-QXYGwgPWNMDvE3kP6SakDrWXbWsNBFKl0V5k7Szpc2XvM_BHiivLA_s5XjGkCuyn9gJdNJR2RDtdWJSC3WOXS2RM0HPQy9iUIlJFiEw',
   'width': 2448}],
 'place_id': 'ChIJ40AO4z05LioRxSnpa66Z96Q',
 'plus_code': {'compound_code': '983X+CF Busselton, Western Australia, Australia',
  'global_code': '4PRQ983X+CF'},
 'rating': 4.5,
 'reference': 'ChIJ40AO4z05LioRxSnpa66Z96Q',
 'scope': 'G

In [14]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in output_data.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row['City Name']
    country = row['Country']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(name_address.url)

    # convert to json
    hotel = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel['results'][0]['name']
        hotel_df.loc[index, "Latitude"] = hotel['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, "Longitude"] = hotel['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, "City"] = city
        hotel_df.loc[index, "Country"] = country
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Hotel Name,Latitude,Longitude,City,Country
0,"Art-Otel' "" Kupecheskiy""",58.6019,99.1937,kodinsk,RU
1,Hotel Vagar,62.0688,-7.28508,sorvag,FO
3,Sunset Bungalows,-19.5283,169.262,isangel,VU
6,Hotel Senac Ilha do Boi,-20.3106,-40.2827,vila velha,BR
7,Ocean Eleven,-34.4236,19.2344,hermanus,ZA
8,Hotel Provincia,16.7388,-92.6377,san cristobal,MX
9,Busselton Beach Resort,-33.6354,115.381,busselton,AU
10,Le Port Hotel,-25.0256,46.9906,taolanaro,MG
11,Crowne Plaza Xiangxi,28.22,109.674,jishou,CN
12,The Rarotongan Beach Resort & Lagoonarium,-21.2574,-159.816,avarua,CK


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [32]:
# Add marker layer ontop of heat map
hotel_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)', 
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5)


fig = gmaps.figure()
hotel_fig.add_layer(hotel_symbol_layer)

hotel_fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Display Figure
combined_fig = gmaps.figure()

combined_fig.add_layer(heat_layer)
combined_fig.add_layer(hotel_symbol_layer)

combined_fig


Figure(layout=FigureLayout(height='420px'))